## Imports

In [1]:
# 3. Import libraries and modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools
from itertools import groupby
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from datetime import datetime, timedelta
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb


np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import image

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from PIL import Image
import cv2

from keras.applications import VGG16


from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K
import tensorflow as tf


%matplotlib inline

/home/yan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [3]:
# Useful methods for plotting results
class HelperClass(object):
        
    # helper method
    @staticmethod
    def stars():
        print ("***********************")
    
    # print metrics as dataframe
    @staticmethod
    def print_dataframe(values):
    
        metrics_print = ['True Positive', 'True Negative', 'False Negative', 'False Positive',
    'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1 Score', 'Roc Auc Score']

        df_metrics = pd.DataFrame(
            {'Metrics': metrics_print,
             'Values': values
            })

        print (df_metrics.to_string(header=False, index=False))
        HelperClass.stars()
        
                
        
    # Disclaimer: metodo obtido em:
    # http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    # Metodo para plotar a matrix de confusao
    @staticmethod
    def plot_confusion_matrix(cm, classes,
                              normalize=False,
                              title='Confusion matrix',
                              cmap=plt.cm.Blues):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix:")
        else:
            print('Confusion matrix, without normalization:')

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        plt.show()




    # ROC curve
    @staticmethod
    def plot_roc_curve(y_true, y_scores):

        fpr, tpr, thresholds = roc_curve(y_true, y_scores)

        HelperClass.stars()
        print ("Roc Curve:")
        HelperClass.stars()

        plt.plot(fpr, tpr, label = 'ROC Curve', linewidth = 2)
        plt.plot([0,1],[0,1], 'k--', linewidth = 2)
        plt.title('ROC Curve')
        plt.xlim([0.0, 0.001])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()



    # Precision Recall Curve
    @staticmethod
    def plot_precision_recall_curve(y_true, y_scores):

        HelperClass.stars()
        print ("Precision Recall Curve:")
        HelperClass.stars()

        precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

        plt.step(recall, precision, color = 'b', alpha = 0.2, where = 'post')

        plt.plot(recall, precision, linewidth=2)
        plt.xlim([0.0,1.0])
        plt.ylim([0.0,1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision Recall Curve')
        plt.show()



    # Show classification report
    @staticmethod
    def show_full_classification_report(y_true, y_pred, y_scores, classes):

        HelperClass.stars()
        print ("Metrics Report:")
        HelperClass.stars()

        cm = confusion_matrix(y_true, y_pred)

        true_positive = cm[1,1]
        true_negative = cm[0,0]
        false_negative = cm[1,0]
        false_positive = cm[0,1]

        accuracy = ((true_positive + true_negative)/(true_positive + true_negative + false_negative + false_positive))
        precision = (true_positive/(true_positive + false_positive))
        recall = (true_positive/(true_positive + false_negative))  
        sensitivity = (true_positive/(true_positive + false_negative))  
        specificity = (true_negative/(true_negative + false_positive))  
        f1_score = ((2 * precision * recall)/(precision + recall))
        
        roc_auc = roc_auc_score(y_true, y_scores)

        print ("Classification Report:")
        HelperClass.stars()

        values = []

        values.append(true_positive)
        values.append(true_negative)
        values.append(false_negative)
        values.append(false_positive)
        values.append('{:.4f}'.format(accuracy))
        values.append('{:.4f}'.format(precision))
        values.append('{:.4f}'.format(recall))
        values.append('{:.4f}'.format(specificity))
        values.append('{:.4f}'.format(f1_score))
        values.append('{:.4f}'.format(roc_auc))

        HelperClass.print_dataframe(values)

        HelperClass.plot_confusion_matrix(cm, classes)

        HelperClass.plot_roc_curve(y_true, y_scores)

        HelperClass.plot_precision_recall_curve(y_true, y_scores)

## Standard

In [4]:
# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print (X_train.shape)

img_rows, img_cols = 28, 28

# 5. Preprocess input data
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
    print (1)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    print (0)
    

print (X_train.shape)
    

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


print (y_train[:10])
 
# Preprocess class labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print (y_train)
print (y_test)
 
# Define model architecture
model = Sequential()
 
model.add(Convolution2D(32, 3, 3, activation='relu', kernel_initializer='glorot_uniform', input_shape=input_shape))
model.add(Convolution2D(32, 3, 3, activation='relu', kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

checkpointer = ModelCheckpoint(filepath='bestmodel.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
 
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])
 
# Fit model on training data
model.fit(X_train, y_train, 
          batch_size=32, nb_epoch=1, verbose=1, callbacks=[checkpointer, earlystopper])
 
# Evaluate model on test data
score = model.evaluate(X_test, y_test, verbose=1)

(60000, 28, 28)
0
(60000, 28, 28, 1)
[5 0 4 1 9 2 1 3 1 4]
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/home/yan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", kernel_initializer="glorot_uniform", input_shape=(28, 28, 1...)`
/home/yan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", kernel_initializer="glorot_uniform")`
/home/yan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/1
  288/10000 [..............................] - ETA: 6s

/home/yan/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/home/yan/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


10000/10000 [==============================] - 5s 516us/step


## ROC

In [ ]:
y_pred = model.predict_proba(X_test)
y_pred_bool = np.zeros_like(y_pred)
y_pred_bool[np.arange(len(y_pred)), y_pred.argmax(1)] = 1

print (roc_auc_score(y_test, y_pred))
print (roc_auc_score(y_test, y_pred_bool))

In [ ]:
def roc_auc_score_multiclass(actual_class, pred_class):
    #creating a set of all the unique classes using the actual class list
    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        #creating a list of all the classes except the current class 
        other_class = [x for x in unique_class if x != per_class]

        #marking the current class as 1 and all other classes as 0
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]

        #using the sklearn metrics method to calculate the roc_auc_score
        roc_auc = roc_auc_score(new_actual_class, new_pred_class)
        roc_auc_dict[per_class] = roc_auc

    return roc_auc_dict

In [ ]:
actual_list = y_test.argmax(axis=1)

In [ ]:
pred_list = y_pred_bool.argmax(axis=1)

In [ ]:
roc_auc_score_multiclass(actual_list, pred_list)

## Standard Roc

In [ ]:
# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print (X_train.shape)

img_rows, img_cols = 28, 28

# 5. Preprocess input data
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
    print (1)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    print (0)
    

print (X_train.shape)
    

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


print (y_train[:10])
 
# Preprocess class labels
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print (y_train)
print (y_test)
 
# Define model architecture
model = Sequential()
 
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=input_shape))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

my_callbacks = [EarlyStopping(monitor='auc_roc', patience=300, verbose=1, mode='max')]
 
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy', auc_roc])
 
# Fit model on training data
model.fit(X_train, y_train, 
          batch_size=32, nb_epoch=2, verbose=1, callbacks=my_callbacks)
 
# Evaluate model on test data
score = model.evaluate(X_test, y_test, verbose=1)

## Load Images

In [ ]:
#https://github.com/ZFTurbo/KAGGLE_DISTRACTED_DRIVER/blob/master/run_keras_simple.py

In [ ]:
import os 
import numpy as np 
from keras.preprocessing import image
 
PATH = os.getcwd()
 
train_path = PATH+'/data/train/'
train_batch = os.listdir(train_path)
x_train = []
 
# if data are in form of images
for sample in train_data:
    img_path = train_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_train.append(x)
    

test_path = PATH+'/data/test/'
test_batch = os.listdir(test_path)
x_test = []
 
for sample in test_data:
    img_path = test_path+sample
    x = image.load_img(img_path)
    # preprocessing if required
    x_test.append(x)

# finally converting list into numpy array
x_train = np.array(x_train)
x_test = np.array(x_test)

## transfer learning

In [ ]:
vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [ ]:
train_dir = './clean-dataset/train'
validation_dir = './clean-dataset/validation'
 
nTrain = 600
nVal = 150

## Pokemon CNN

In [ ]:
img_path = '/home/yan/dataset/bulbasaur/00000000.png'

img = cv2.imread(img_path)
print (img.shape)

plt.imshow(img)



'''
cv2.imshow("Input", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

In [8]:
import os 
import numpy as np 
from keras.preprocessing import image
 

train_path = '/home/yan/dataset/'
list_folders = os.listdir(train_path)

print (list_folders)

# initialize the data and labels
X_train = []
y_train = []
 
# if data are in form of images
for folder in list_folders:
    full_folder = train_path + folder
    #print (full_folder)
    for img in os.listdir(full_folder):
        img_path = (full_folder + '/' + img)
        #print (img_path)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (900, 900))
        image = image.astype('float32')
        image = image / 255
        image = img_to_array(image)
        
        X_train.append(image)      
        y_train.append(folder)

['mewtwo', 'charmander', 'pikachu', 'bulbasaur', 'squirtle']


In [9]:
X_train = np.array(X_train)
X_train.shape

(427, 900, 900, 3)

In [36]:
print (X_train[3].shape)

(900, 900, 3)


In [37]:
# binarize the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)

In [38]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(X_train, X_test, y_train, y_test) = train_test_split(X_train,
    y_train, test_size=0.2, random_state=42)

In [7]:
# Define model architecture
model = Sequential()

model.add(Convolution2D(32, (3, 3), padding="same", kernel_initializer='glorot_uniform', input_shape=(900, 900, 3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3), kernel_initializer='glorot_uniform', padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, (3, 3), kernel_initializer='glorot_uniform', padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3, 3), kernel_initializer='glorot_uniform', padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(128, (3, 3), kernel_initializer='glorot_uniform', padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# softmax classifier
model.add(Dense(5))
model.add(Activation("softmax"))

In [6]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, vertical_flip=True, fill_mode="nearest")

In [40]:
checkpointer = ModelCheckpoint(filepath='bestmodel.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
 
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])
 
# Fit model on training data
model.fit(X_train, y_train, 
          batch_size=32, nb_epoch=1, verbose=1, callbacks=[checkpointer, earlystopper])
 
# Evaluate model on test data
score = model.evaluate(X_test, y_test, verbose=1)

/home/yan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 341 arrays: [array([[[0.10980392, 0.00392157, 0.15686275],
        [0.10980392, 0.00392157, 0.15686275],
        [0.11764706, 0.        , 0.16078432],
        ...,
        [0.13333334, 0.        , 0.16078432],
  ...